In [1]:
from pyspark import SparkContext
import pyspark

conf = pyspark.SparkConf().set("spark.driver.host", "localhost")
sc = SparkContext.getOrCreate()
sc.setLogLevel('OFF')

21/12/08 15:16:06 WARN Utils: Your hostname, Fans-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 10.0.0.115 instead (on interface en0)
21/12/08 15:16:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/08 15:16:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/08 15:16:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
supervisor_sf_pair = sc.textFile("../data/SF_business/supervisor_sf.tsv")\
                       .map(lambda x : x.split("\t"))\
                       .map(lambda x : (x[0], x[1]))

In [3]:
zip_biz_pair = sc.textFile("../data/SF_business/filtered_registered_business_sf.tsv")\
                 .map(lambda x : x.split("\t"))\
                 .map(lambda x : (x[0], x[1:]))

## 1. Return any five distinct  zip code in zip_biz_pair

In [4]:
zip_biz_pair.keys().distinct().take(5)

['94124', '94108', '94102', '94133', '94111']

## 2. Return any five distinct city names.

In [5]:
zip_biz_pair.values().map(lambda x : x[-2]).distinct().take(5)

['San Francisco', 'Hayward', 'Redwood+city', 'Daly+city', 'Mexico']

## 3. Return five distinct (zip code, city name) pairs ordered by zip code, where zip code has 5 digits.

In [5]:
zip_biz_pair.filter(lambda x: len(x[0])==5).mapValues(lambda x: x[2]).distinct().sortByKey().take(5)

[('10001', 'New+york'),
 ('10002', 'New+york'),
 ('10003', 'New+york'),
 ('10004', 'New+york'),
 ('10004', 'New York')]

In [6]:
distinct_zip_city_pairs = zip_biz_pair.filter(lambda x : len(x[0]) == 5)\
                                      .mapValues(lambda x : x[-2])\
                                      .distinct()\
                                      .sortByKey()

In [7]:
distinct_zip_city_pairs.take(5)

[('10001', 'New+york'),
 ('10002', 'New+york'),
 ('10003', 'New+york'),
 ('10004', 'New+york'),
 ('10004', 'New York')]

## 4. Find the zip associated with the most city names
* which one is more efficient ??
    - zip_associated_with_multiple_cities_1  $vs$ zip_associated_with_multiple_cities_2

In [21]:
distinct_zip_city_pairs.mapValues(lambda x: 1).reduceByKey(lambda x,y: x+y).sortBy(lambda x: x[1], ascending=False).top(1,key=lambda x: x[1])

[('94080', 40)]

#### v1

In [8]:
zip_associated_with_multiple_cities_1 = distinct_zip_city_pairs.map(lambda x : (x[0], 1))\
                                           .groupByKey()\
                                           .mapValues(lambda x : sum(x))\
                                           

In [9]:
zip_associated_with_multiple_cities_1.sortBy(lambda x : x[1], ascending = False).take(1)

[('94080', 40)]

#### v2

In [10]:
zip_associated_with_multiple_cities_2 = distinct_zip_city_pairs.map(lambda x : (x[0], 1))\
                                           .reduceByKey(lambda x, y : x + y)\
                                           

In [11]:
zip_associated_with_multiple_cities_2.sortBy(lambda x : x[1], ascending = False).take(1)

[('94080', 40)]

### Which one is more efficient? v1 or v2??

#### 

## 5. Create zip_city_count_names which includes (zip, (count, the list of names))

In [33]:
distinct_zip_city_pairs.mapValues(lambda x: (1,[x])).reduceByKey(lambda x,y: (x[0]+y[0],sorted(x[1]+y[1])))\
                                                    .sortBy(lambda x: x[1][0],ascending=False).take(1)

[('94080',
  (40,
   ['Daly City',
    'S San Fran',
    'S San Francisco',
    'S+san+fran',
    'S+san+francisco',
    'S. San Francisco',
    'S.+san+francisco',
    'San Francisco',
    'San+bruno',
    'San+carlos',
    'San+francisco',
    'So San Francisco',
    'So+.+san+francisco',
    'So+san+francisco',
    'So.+san+francisco',
    'So.san+francisco',
    'Soith+san+francisco',
    'Souht+san+francisco',
    'Sout+san+francisco',
    'South San Francisco',
    'South San Fransico',
    'South Sanfrancisco',
    'South++san+francisco',
    'South+san+fracisco',
    'South+san+fran',
    'South+san+franc',
    'South+san+francisc',
    'South+san+francisco',
    'South+san+francisico',
    'South+san+francsico',
    'South+san+franisco',
    'South+san+franscisco',
    'South+san+fransisco',
    'South+san.francisco',
    'South+sanfrancisco',
    'South+sanfranciso',
    'South+sanfransico',
    'South+sf',
    'Ssf',
    'Suth+san+francisco']))]

In [42]:
zip_city_count_names = distinct_zip_city_pairs.mapValues(lambda x: [x])\
    .reduceByKey(lambda x, y: x + y)\
    .map(lambda x: (x[0], (len(list(x[1])), sorted(list(x[1])))))\
    .sortBy(lambda x: x[1][0], ascending=False)

In [43]:
zip_city_count_names.take(1)

[('94080',
  (40,
   ['Daly City',
    'S San Fran',
    'S San Francisco',
    'S+san+fran',
    'S+san+francisco',
    'S. San Francisco',
    'S.+san+francisco',
    'San Francisco',
    'San+bruno',
    'San+carlos',
    'San+francisco',
    'So San Francisco',
    'So+.+san+francisco',
    'So+san+francisco',
    'So.+san+francisco',
    'So.san+francisco',
    'Soith+san+francisco',
    'Souht+san+francisco',
    'Sout+san+francisco',
    'South San Francisco',
    'South San Fransico',
    'South Sanfrancisco',
    'South++san+francisco',
    'South+san+fracisco',
    'South+san+fran',
    'South+san+franc',
    'South+san+francisc',
    'South+san+francisco',
    'South+san+francisico',
    'South+san+francsico',
    'South+san+franisco',
    'South+san+franscisco',
    'South+san+fransisco',
    'South+san.francisco',
    'South+sanfrancisco',
    'South+sanfranciso',
    'South+sanfransico',
    'South+sf',
    'Ssf',
    'Suth+san+francisco']))]

## 6. convert zip_city_count_names to (zip, city_name)

In [48]:
zip_city_count_names.flatMapValues(lambda x: x[1]).take(5)

[('94080', 'Daly City'),
 ('94080', 'S San Fran'),
 ('94080', 'S San Francisco'),
 ('94080', 'S+san+fran'),
 ('94080', 'S+san+francisco')]

In [14]:
zip_city_count_names.flatMapValues(lambda x: x[1]).collect()

[('94080', 'Daly City'),
 ('94080', 'S San Fran'),
 ('94080', 'S San Francisco'),
 ('94080', 'S+san+fran'),
 ('94080', 'S+san+francisco'),
 ('94080', 'S. San Francisco'),
 ('94080', 'S.+san+francisco'),
 ('94080', 'San Francisco'),
 ('94080', 'San+bruno'),
 ('94080', 'San+carlos'),
 ('94080', 'San+francisco'),
 ('94080', 'So San Francisco'),
 ('94080', 'So+.+san+francisco'),
 ('94080', 'So+san+francisco'),
 ('94080', 'So.+san+francisco'),
 ('94080', 'So.san+francisco'),
 ('94080', 'Soith+san+francisco'),
 ('94080', 'Souht+san+francisco'),
 ('94080', 'Sout+san+francisco'),
 ('94080', 'South San Francisco'),
 ('94080', 'South San Fransico'),
 ('94080', 'South Sanfrancisco'),
 ('94080', 'South++san+francisco'),
 ('94080', 'South+san+fracisco'),
 ('94080', 'South+san+fran'),
 ('94080', 'South+san+franc'),
 ('94080', 'South+san+francisc'),
 ('94080', 'South+san+francisco'),
 ('94080', 'South+san+francisico'),
 ('94080', 'South+san+francsico'),
 ('94080', 'South+san+franisco'),
 ('94080', 'S

In [49]:
sc.stop()